In [ ]:
# 패키지 설치
!pip install pickle5 --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle5 as pickle
sns.set()

In [ ]:
# 가격 데이터 로드
with open('equity_price.pkl', 'rb') as f:
    price = pickle.load(f)

In [ ]:
price

In [ ]:
# 주식 모멘텀 클래스
class EquityMomentum:

    # 초기화 함수
    def __init__(self, price, lookback_period, n_selection, cost=0.0005, signal_method='dm', long_only=False):

        # 수익률
        self.rets = price.pct_change().fillna(0)

        # 모멘텀 방식 선택
        if signal_method == 'am':
            self.signal = self.absolute_momentum(price, lookback_period, long_only)
        elif signal_method == 'rm':
            self.signal = self.relative_momentum(price, lookback_period, n_selection, long_only)
        elif signal_method == 'dm':
            self.signal = self.dual_momentum(price, lookback_period, n_selection, long_only)

        # 거래비용
        self.cost = cost
        
        # 포트폴리오 수익률
        self.port_rets = self.calculate_returns(self.rets, self.signal, self.cost)

        # 샤프비율
        self.sharpe_ratio = self.calculate_sharpe_ratio(self.port_rets)

        # 백테스팅 결과
        self.plot_result(self.port_rets)

    # 절대 모멘텀 시그널 계산 함수
    def absolute_momentum(self, price, lookback, long_only=False):

        # N일 수익률  
        returns = price.pct_change(periods=lookback)

        # 롱 시그널
        long_signal = (returns > 0) * 1

        # 숏 시그널
        short_signal = (returns < 0) * -1

        # 토탈 시그널
        if long_only:
            signal = long_signal
        else:
            signal = long_signal + short_signal
        
        return signal
    
    # 상대 모멘텀 시그널 계산 함수
    def relative_momentum(self, price, lookback, n_selection, long_only=False):

        # N일 수익률
        returns = price.pct_change(periods=lookback)

        # 수익률 순위화
        rank = returns.rank(axis=1, ascending=False)

        # 롱 시그널
        long_signal = (rank <= n_selection) * 1

        # 숏 시그널
        short_signal = (rank >= len(rank.columns) - n_selection + 1) * -1

        # 토탈 시그널
        if long_only:
            signal = long_signal
        else:
            signal = long_signal + short_signal

        return signal
    
    # 듀얼 모멘텀 시그널 계산 함수
    def dual_momentum(self, price, lookback, n_selection, long_only=False):

        # 절대 모멘텀 시그널
        abs_signal = self.absolute_momentum(price, lookback, long_only)

        # 상대 모멘텀 시그널
        rel_signal = self.relative_momentum(price, lookback, n_selection, long_only)

        # 듀얼 모멘텀 시그널
        signal = (abs_signal == rel_signal) * abs_signal

        return signal

    # 포트폴리오 수익률 계산 함수
    def calculate_returns(self, rets, signals, cost):

        # 포트폴리오 수익률
        port_rets = signals.shift() * rets - abs(signals.diff()) * cost

        return port_rets.mean(axis=1)

    # 샤프비율 계산
    def calculate_sharpe_ratio(self, total_returns):
        sharpe_ratio = total_returns.mean() * np.sqrt(252) / total_returns.std()

        return sharpe_ratio

    # 백테스팅 결과 시각화 함수
    def plot_result(self, rets):

        # 백테스팅 결과 시각화
        plt.figure(figsize=(12, 6))
        rets.cumsum().plot()
        plt.show()

### 패러미터 세팅

In [ ]:
# 룩백 윈도우
lookback = 20 * 24

# 상대 모멘텀 롱숏 개수
n_selection = len(price.columns) / 4

In [ ]:
# 주식 모멘텀 백테스팅
momentum = EquityMomentum(price, lookback, n_selection, long_only=True)

In [ ]:
momentum.sharpe_ratio

Copyright 2022. 퀀트대디. All rights reserved.